## Test of the processing pipeline that will take place for every metric
> This file's purpose is to **test** the new approach we will be using to complete the project.

In [ ]:



# Add cwd/.. to path so the imports to user defined packages work
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))
from src.data_fetchers.metrics_fetcher import MetricsFetcher
from src.data_fetchers.classification_fetcher import ClassificationFetcher
from src.data_fetchers.returns_fetcher import ReturnsFetcher
from src.configs.file_style_configs_by_metric import file_style_configs_by_metric


## Generative question:
- How does investment criteria affect the returns of a portfolio?

### Hypothesis:
- Some metrics or percentiles of them will yield portfolios with higher returns than the SP500 consistently over time.

### Steps:
1. [Load data for a single metric](#step-1)
2. Limit span of data to 2000Q1 to 2023Q2
3. Plot distribution of data. We need to make sure data was fetched correctly
4. Clean data if needed (outliers, missing values, etc. Maybe calculate No. of missing values, etc.)
5. Save cleaned data (.pickle)
6. Select study and investment periods
7. Calculate average of metric per company
8. Calculate central tendency and dispersion measures for the averages
9. Create portfolios of companies based percentiles on the average (or median) of metric
10. Calculate returns of portfolios and SP500
11. Compute the difference in returns between portfolios and SP500
12. Test for statistical significance of the difference in returns
13. Evaluate risk of portfolios (Maybe classify them based on sortino and sharpe ratios)
14. Evaluate the sensitivity of methodology to changes in the study and investment periods

### Step 1